In [33]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [27]:
songs = pd.read_csv('songs.csv').dropna().reset_index()[['year', 'rank', 'title', 'artist']]
songs

,year,rank,title,artist
0,1946,1 (1),Prisoner of Love,Perry Como
1,1946,2 (2),To Each His Own,Eddy Howard
2,1946,3 (3),The Gypsy,The Ink Spots
3,1946,4 (4),Five Minutes More,Frank Sinatra
4,1946,5 (5),Rumors Are Flying,Frankie Carle
...,...,...,...,...
6851,2022,96\n,Flower Shops,Ernest
6852,2022,97\n,To the Moon,Jnr Choi
6853,2022,98\n,Unholy,Sam Smith
6854,2022,99\n,One Mississippi,Kane Brown


In [29]:
songs['rank'] = songs['rank'].apply(lambda x: x.split()[0])
songs['rank'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', 'Tie', '51', '52', '53', '54', '55',
       '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66',
       '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77',
       '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
       '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99',
       '100'], dtype=object)

In [32]:
songs = songs[songs['rank']!='Tie'].reset_index()[['year', 'rank', 'title', 'artist']]
songs['rank'] = songs['rank'].astype(int)
songs

,year,rank,title,artist
0,1946,1,Prisoner of Love,Perry Como
1,1946,2,To Each His Own,Eddy Howard
2,1946,3,The Gypsy,The Ink Spots
3,1946,4,Five Minutes More,Frank Sinatra
4,1946,5,Rumors Are Flying,Frankie Carle
...,...,...,...,...
6849,2022,96,Flower Shops,Ernest
6850,2022,97,To the Moon,Jnr Choi
6851,2022,98,Unholy,Sam Smith
6852,2022,99,One Mississippi,Kane Brown


In [34]:
def get_search_url(title):
    query = ''
    parts = title.split()
    query = '+'.join(parts)
    return f"https://www.lyricfinder.org/search?searchtype=artists&query={query}"

In [81]:
def find_lyrics(row):
    global counter
    lyrics = row['lyrics']
    
    if counter == 100 and lyrics != "NEXT_TIME":
        return "NEXT_TIME"
    elif counter == 100:
        return lyrics
    
    if lyrics != "NEXT_TIME":
        return row['lyrics']
    title = row['title']
    artist_full = row['artist']
    artist = artist_full.split()[0].lower()
    print(f'{counter}) Trying to find {artist_full} - {title}')
    counter += 1
    resp = requests.get(get_search_url(title))
    if (resp.status_code != 200):
        print(f'** Failure! Status code: {resp.status_code} **')
        return 'HTTP_ERROR'
    
    bs = BeautifulSoup(resp.text, 'html.parser')
    lines = bs.select("div div div a")
    link = None
    for line in lines:
        line_artist = line.select_one('h4 span').text.lower()
        if artist in line_artist:
            link = line.get('href')
            break
    if link == None:
        print(f'** Song Not Found! **')
        return 'NOT_FOUND'
    
    resp = requests.get("http:" + link)
    if (resp.status_code != 200):
        print(f'** Failure! Status code: {resp.status_code} **')
        return 'HTTP_ERROR'
    
    bs = BeautifulSoup(resp.text, 'html.parser')
    lyrics_div = bs.select_one('.col-lg-6')
    h3tag = lyrics_div.find('h3')
    h3tag.extract()
    return lyrics_div.text


In [ ]:
songs['lyrics'] = "NEXT_TIME"

In [93]:
songs = pd.read_csv('songs_with_lyrics.csv')[['year', 'rank', 'title', 'artist', 'lyrics']]

songs.to_csv('songs_with_lyrics.csv')

In [92]:
for i in range(5):
    counter = 0
    songs = pd.read_csv('songs_with_lyrics.csv')[['year', 'rank', 'title', 'artist', 'lyrics']]
    songs['lyrics'] = songs.apply(find_lyrics, axis=1)
    songs.to_csv('songs_with_lyrics.csv')

0) Trying to find Elvis Presley - Good Luck Charm
1) Trying to find Kenny Ball - Midnight in Moscow
2) Trying to find Tommy Roe - Sheila
3) Trying to find Sam Cooke - Twistin' the Night Away
4) Trying to find The Orlons - The Wah-Watusi
5) Trying to find Joey Dee and the Starliters - Peppermint Twist
6) Trying to find Brenda Lee - Break It to Me Gently
7) Trying to find The Marvelettes - Playboy
8) Trying to find Nat King Cole - Ramblin' Rose
9) Trying to find Brian Hyland - Sealed with a Kiss
10) Trying to find Jay and the Americans - She Cried
11) Trying to find Connie Francis - Don't Break the Heart That Loves You
12) Trying to find Gary U.S. Bonds - Dear Lady Twist
** Song Not Found! **
13) Trying to find Sue Thompson - Norman
14) Trying to find Ketty Lester - Love Letters
15) Trying to find Claudine Clark - Party Lights
16) Trying to find The Highwaymen - Cotton Fields
17) Trying to find Bent Fabric - Alley Cat
** Failure! Status code: 404 **
18) Trying to find The Isley Brothers 

In [90]:
df = pd.read_csv("topSongsLyrics1950_2019.csv").dropna().reset_index()[['year', 'rank', 'song', 'artist', 'lyrics']]
df['lyrics'] = df['lyrics'].apply(lambda x: x.replace('|', '\n'))
df.rename(columns={'song': 'title'}, inplace=True)
df

,year,rank,title,artist,lyrics
0,1950,1,The Fat Man,Fats Domino,"They call, they call me the fat man\n'Cause I ..."
1,1950,2,Please Send Me Someone To Love,Percy Mayfield,"Understanding and peace of mind\nBut, if it's ..."
2,1950,3,Teardrops From My Eyes,Ruth Brown,I think of you\nAnd that's the time\nI feel so...
3,1950,4,Mona Lisa,Nat King Cole,"Mona Lisa, Mona Lisa, men have named you\nYou'..."
4,1950,5,Tennessee Waltz,Patti Page,When an old friend I happened to see\nI Introd...
...,...,...,...,...,...
691,2019,6,Sucker,Jonas Brothers,We go together\nBetter than birds of a feather...
692,2019,7,7 rings,Ariana Grande,"Yeah, breakfast at Tiffany's and bottles of bu..."
693,2019,8,Highest in the Room,Travis Scott,I got room in my fumes (Yeah)\nShe fill my min...
694,2019,9,Please Me,Cardi B and Bruno Mars,"Uh, uh, yeah, come on\nPlease me, baby\nTurn a..."


In [91]:
dataset = pd.concat([df, songs])[['year', 'rank', 'title', 'artist', 'lyrics']]
dataset = dataset.drop_duplicates(subset=['title', 'artist'])
dataset.to_csv("songs_with_lyrics.csv")
dataset

,year,rank,title,artist,lyrics
0,1950,1,The Fat Man,Fats Domino,"They call, they call me the fat man\n'Cause I ..."
1,1950,2,Please Send Me Someone To Love,Percy Mayfield,"Understanding and peace of mind\nBut, if it's ..."
2,1950,3,Teardrops From My Eyes,Ruth Brown,I think of you\nAnd that's the time\nI feel so...
3,1950,4,Mona Lisa,Nat King Cole,"Mona Lisa, Mona Lisa, men have named you\nYou'..."
4,1950,5,Tennessee Waltz,Patti Page,When an old friend I happened to see\nI Introd...
...,...,...,...,...,...
6998,2019,1,Old Town Road (Remix),Lil Nas X (feat. Billy Ray Cyrus),NEXT_TIME
6999,2019,3,Señorita,Shawn Mendes & Camila Cabello,NEXT_TIME
7000,2019,5,I Don't Care,Ed Sheeran and Justin Bieber,NEXT_TIME
7001,2019,7,7 rings,Ariana Grande,NEXT_TIME
